In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
from huggingface_hub import login

In [2]:
# load model for correcting OCR errors, 2 models available
peft_model_id = "mariannam/llammas-OCR-FT13k"
config = PeftConfig.from_pretrained(peft_model_id)

adapter_config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

In [3]:
# load base model
base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    device_map="auto", # for CPU use device_map=None
    torch_dtype="auto"
)

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/161 [00:00<?, ?B/s]

In [4]:
# load adapters
model = PeftModel.from_pretrained(base_model, peft_model_id)


adapter_model.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

In [5]:
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [39]:
# input ocr text to fix HERE
ocr_text = """
Sendadele.

Külm ja waitkne wenna Jüda
Iga mees jo teab feda;
Uga fJageft waikjel weel
Siäraltne keel, |
Keda fee ainult mõistab
Kellesje fee afi kostab.
Nõnda menna füda kaa
Kaiifelt maln tunmiga
Venda jumalaga jätab,
Kes nii kaugel” ära tõttab.
Peehe watm, kes targalt mõistab,
Kohus- laugelt wastu kostab,
Et ei muu parem nõu

"""


In [40]:
# prompt tempelate
prompt = f"""### Instruction:
Paranda vead selles eestikeelses OCR tekstis.

### Input:
{ocr_text}

### Response:
"""

In [41]:
# calculate dynamic max new tokens allowed
input_length = len(tokenizer(ocr_text, truncation=True)['input_ids'])
max_new_tokens = input_length + 10


In [42]:
max_new_tokens

176

In [43]:
# generate and print output
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

In [44]:
outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
print(tokenizer.decode(outputs[0], skip_special_tokens=True)[len(prompt):].strip())

Sendadele.

Külm ja waikne wennad Jüda
Iga mees ja teab seda;
Uga fJageft waikjel weel
Siäraltne keel, |
Keda fee ainult mõistab
Kellesje fee afi kostab.
Nõnda menna füda kaa
Kaifelt maln tunmiga
Venda jumalaga jätab,
Kes nii kaugel” ära tõttab.
Peehe watm, kes targalt mõistab,
Kohus laugelt wastu kostab,
Et ei muu parem nõu
